In [2]:
!pip3 install transformers

In [3]:
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

In [4]:
text_generator=pipeline("text-generation",model="distilbert/distilgpt2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
generated_text=text_generator("Today is a rainy day",truncation=True,num_return_sequences=2)

In [24]:
print(generated_text[0]['generated_text'])

Today is a rainy day and there are several days when the snow falls into the snow.


So we had to do something about this.
It was the snow that made the snow fall the hardest to avoid.
In the winter, the snow fell in the middle of the night. The snow fell on the roof of the house.
At the time, the snow was almost falling on the roof of the house and in the backyard.
The snow was only about 12 inches.
So the snow wasn't falling on the roof of the house.
But the snow fell in the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell in the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The snow fell into the house.
The

In [7]:
## Question and Answering

In [25]:
qa_model=pipeline("question-answering")
question="what is my job"
conext="I am developing something"
qa_model(question=question,context=conext)

No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu


{'score': 0.508809506893158,
 'start': 5,
 'end': 25,
 'answer': 'developing something'}

In [9]:
## Tokenization

In [26]:
from transformers import AutoTokenizer,DistilBertTokenizer,AutoTokenizer,DistilBertForSequenceClassification,AutoModelForSequenceClassification

In [27]:
model_name1="distilbert-base-uncased-finetuned-sst-2-english"
mytokenizer1=DistilBertTokenizer.from_pretrained(model_name1)
mymodel1=DistilBertForSequenceClassification.from_pretrained(model_name1)
classifier=pipeline("sentiment-analysis")
res=classifier("I am not so happy")
print(res)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


[{'label': 'NEGATIVE', 'score': 0.9997592568397522}]


In [12]:
# model_name2="nlptown/bert-base-multilingual-uncased-sentiment"
# mymodel2=AutoModelForSequenceClassification.from_pretrained(model_name2)
# mytokenizer=AutoTokenizer.from_pretrained(mymodel2)
# classifier=pipeline("sentiment-analysis",model=mymodel2,tokenizer=mytokenizer)
# res=classifier("Hello")
# print(res)

In [28]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
text="I was not so happy"
tokens=tokenizer.tokenize(text)
print(tokens)
input_ids=tokenizer.convert_tokens_to_ids(tokens)
encoded_output=tokenizer(text)
print(encoded_output)
decode_output=tokenizer.decode(input_ids)
print(decode_output)

['i', 'was', 'not', 'so', 'happy']
{'input_ids': [101, 1045, 2001, 2025, 2061, 3407, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}
i was not so happy


In [14]:
## Dataset

In [29]:
!pip3 install datasets

In [30]:
from datasets import load_dataset

In [31]:
dataset=load_dataset("imdb")

In [18]:
## preprocess the data

In [32]:
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

In [33]:
def tokenize_function(examples):
    return tokenizer(examples['text'],padding="max_length",truncation=True)
tokenized_datasets=dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [34]:
!pip3 install accelerate -U

In [35]:
from transformers import TrainingArguments
training_args=TrainingArguments(output_dir='./results',eval_strategy="epoch",learning_rate=2e-5,per_device_eval_batch_size=16,num_train_epochs=3,weight_decay=0.01)

In [39]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    # remove evaluation_strategy and save_strategy if unsupported
)


In [ ]:
from transformers import AutoModelForSequenceClassification
model=AutoModelForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)
trainer=Trainer(model=model,args=training_args,train_dataset=tokenized_datasets['train'],eval_dataset=tokenized_datasets['test'])
# trainer=Trainer(model=model,args=training_args,train_dataset=tokenized_datasets['train'],eval_dataset=tokenized_datasets['test'])
trainer.train()
## evaluate the model
results=trainer.evaluate()
print(results)
model.save_pretrained('./fine-tune-model')
tokenizer.save_pretrained('./fine-tuned-model')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saisowhitp (saisowhitp-no) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss


In [1]:
!pip3 install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.0 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=bde9c21bed4ef1a7d64dfb99e64aad9c141359ec35d7b192b4d00b9d02f5bce5
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [2]:
from transformers import pipeline
import warnings
warnings.filterwarnings('ignore')

In [3]:
import arxiv
import pandas as pd
query='ai OR Artifical Intelligence'
search=arxiv.Search(query=query,max_results=10,sort_by=arxiv.SortCriterion.SubmittedDate)
papers=[]
for result in search.results():
    papers.append({'published':result.published,'title':result.title,'abstract':result.summary,'categories':result.categories})


    df=pd.DataFrame(papers)
df.head()
abstract=df['abstract'][0]
summarizer=pipeline("summarization",model="facebook/bart-large-cnn")
summarization_result=summarizer(abstract)
summarization_result[0]['summary_text']

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


'Mini-o3 is a system that executes deep, multi-turn reasoning. It achieves state-of-the-art performance on challenging visual search tasks. Despite training with an upper bound of only six interaction turns, our model generates trajectories that naturally scale to tens of turns at inference time.'